# Загрузка открытых данных с dtp-stat.ru
Это подготовительный блокнот для загрузки и объеденения данных.

См [блокнот с анализом](1-Dtp-analysis.ipynb)

## Загрузка данных в локальные файлы

In [ ]:
import requests as r
import os
import aiohttp
from bs4 import BeautifulSoup as bs

links = []
resp = r.get('https://dtp-stat.ru/opendata/')
if resp.ok:
    soup = bs(resp.text, "html.parser")
    for link in soup.findAll('a'):
        url = link.get('href')
        if '.geojson' in url: links.append('https://dtp-stat.ru' + url)
if len(links) == 85:
    print('Все регионы присутствуют на сайте. Загружаем')
else:
    print("Some regions are missed")
async with aiohttp.ClientSession() as session:
    for link in links:
        async with session.get(link) as resp:
            file_name = os.path.basename(link)
            with open('data/dtp-stat/' + file_name, 'w') as f:
                f.write(await resp.text())
                print('.', end='')
print(' готово.', end='')

## Склейка файлов в один большой
Склеиваем все скачанные geojson файлы в один большой. Понадобится *много*(~15gb) памяти.
Этот шаг стоит выполнить если вы собираетесь анализировать или передовать весь массив данных сразу.

In [ ]:
import glob

files = glob.glob('data/dtp-stat/*')
import json

features = []
for file in files:
    with open(file) as f:
        d = json.load(f)
        features.extend(d['features'])
result = {
    "type": "FeatureCollection",
    "features": features
}
with open('data/dtp-stat/russia.geojson', 'w') as f:
    json.dump(result, f, ensure_ascii=False)
del result
del features